# 貪欲算法

In [1]:
# 貪欲計法による算出が下界となる
items = {1,2,3,4,5}
c = {1:50, 2:40, 3:10, 4:70, 5:55}
w = {1:7, 2:5, 3:1, 4:9, 5:6}
capacity = 15

# 効率を計算
ratio = {j:c[j]/w[j] for j in items}
ratio

{1: 7.142857142857143,
 2: 8.0,
 3: 10.0,
 4: 7.777777777777778,
 5: 9.166666666666666}

In [2]:
# 効率が良い順を取得
sItems = [key for key, val in sorted(ratio.items(), key=lambda x:x[1], reverse=True)]
sItems

[3, 5, 2, 4, 1]

In [3]:
# すべて0とする
x={j:0 for j in sItems}

# capacityを設定
cap = capacity

# 効率が良い順にループ
for j in sItems:
    # もし入る余裕があるなら、
    if w[j] <= cap:
        # capacityを更新する
        cap -= w[j]
        # 選択する
        x[j] = 1
print(x)
print('総価格 = ', sum(c[j]*x[j] for j in sItems))

{3: 1, 5: 1, 2: 1, 4: 0, 1: 0}
総価格 =  105


# 線形緩和問題

In [4]:
# 線形緩和問題が上界となる
x={j:0 for j in sItems}
cap = capacity

for j in sItems:
    if w[j] <= cap:
        cap -= w[j]
        x[j] = 1
    
    # capを超えた場合はぎりぎりまで足す
    else:
        x[j] = cap/w[j]
        break
print(x)
print('総価格 = ', sum(c[j]*x[j] for j in sItems))

{3: 1, 5: 1, 2: 1, 4: 0.3333333333333333, 1: 0}
総価格 =  128.33333333333334


# 分岐限定法

In [5]:
# 品物をセットで表していく
# セットは引き算が可能
{1,2,3}-{2}

{1, 3}

In [6]:
# unionを使うと外部結合となる
a = {1,2,3}
a.union({2,3,4,7})

{1, 2, 3, 4, 7}

In [7]:
class KnapsackProblem(object):
    """ The difinition of KnapSackProblem """    
    def __init__(self, name, capacity, items, costs, weights,
                 zeros=set(), ones=set()):
        self.name = name
        self.capacity = capacity
        self.items = items
        self.costs = costs
        self.weights = weights
        self.zeros = zeros # ナップサックに入れない品物の集合
        self.ones = ones # ナップサックに入れる品物の集合
        self.lb = -100
        self.ub = -100
        ratio = {j:costs[j]/weights[j] for j  in items}
        self.sitemList =  [k for k, v in 
            sorted(ratio.items(), key=lambda x:x[1], reverse=True)] # 品物を効率の大きい順に並べたリスト
        self.xlb = {j:0 for j in self.items} # lb を達成する解
        self.xub = {j:0 for j in self.items} # ub を達成する解
        self.bi = None # ubを達成する際に値が分数であるもの

    def getbounds(self):
        """ Calculate the upper and lower bounds. """
        # ナップサックに入れない品物を0に変更
        for j in self.zeros:
            self.xlb[j] = self.xub[j] = 0
        # ナップサックに入れる品物を1に変更
        for j in self.ones:
            self.xlb[j] = self.xub[j] = 1
        # もしナップサックに入れる品物だけでcapacityを超えた場合は計算終了
        if self.capacity < sum(self.weights[j] for j in self.ones):
            self.lb = self.ub =  -100
            return 0
        
        # 決定していない品物のセット
        ritems = self.items - self.zeros - self.ones
        # ritemsを効率の良い順に並べたリスト
        sitems = [j for j in self.sitemList if j in ritems]
        # 残りの容量を計算
        cap = self.capacity - sum(self.weights[j] for j in self.ones)
        # 効率の良い順にループ
        for j in sitems:
            # もし容量に余裕があったら、1に変更
            if self.weights[j] <= cap:
                cap -= self.weights[j]
                self.xlb[j] = self.xub[j] = 1
            # 容量に余裕がない場合は ub は容量分だけ追加
            # lbの計算のために次のループに移動
            else:
                self.xub[j] = cap/self.weights[j]
                self.bi = j # biを更新
                break
        # lb,ubを計算
        self.lb = sum(self.costs[j]*self.xlb[j] for j in self.items)
        self.ub = sum(self.costs[j]*self.xub[j] for j in self.items)

    def __str__(self):
        """ KnapSackProblem の情報を印字 """
        return('Name = '+self.name+', capacity = '+str(self.capacity)+',\n'
            'items = '+str(self.items)+',\n'+
            'costs = '+str(self.costs)+',\n'+
            'weights = '+str(self.weights)+',\n'+
            'zeros = '+str(self.zeros)+', ones = '+str(self.ones)+',\n'+
            'lb = '+str(self.lb)+', ub = '+str(self.ub)+',\n'+
            'sitemList = '+str(self.sitemList)+',\n'+
            'xlb = '+str(self.xlb)+',\n'+'xub = '+str(self.xub)+',\n'+
            'bi = '+str(self.bi)+'\n')

In [8]:
from pulp import *

def KnapsackProblemSolve(capacity, items, costs, weights):
    # dequeを使うとリストよりも計算コストが少なく、先頭と末尾に追加可能
    from collections import deque
    queue = deque()
    
    # 問題を設定する
    root = KnapsackProblem('KP', capacity = capacity,
        items = items, costs = costs, weights = weights,
        zeros = set(), ones = set())
    # 下界、上界を計算する
    root.getbounds()
    best = root
    # 問題を追加する
    queue.append(root)
    
    # queueが空になるまで
    while queue != deque([]):
        # 一番左を選択 
        p = queue.popleft()
        p.getbounds()
        # bestのlb よりも pのubが大きい場合(bestを更新する可能性がある)
        if p.ub > best.lb:
            # もしpのlbがbestのlbよりも大きければ更新
            if p.lb > best.lb:
                best = p
            # pのubがpのlbよりも大きければ子問題を作って分岐する
            if p.ub > p.lb:
                # ubを求める際に分数になった番号をkに格納
                k = p.bi
                # kを1,0に分割し、ququeに追加
                p1 = KnapsackProblem(p.name+'+'+str(k),
                    capacity = p.capacity, items = p.items,
                    costs = p.costs, weights = p.weights,
                    zeros = p.zeros, ones = p.ones.union({k}))
                queue.append(p1)
                p2 = KnapsackProblem(p.name+'-'+str(k),
                    capacity = p.capacity, items = p.items,
                    costs = p.costs, weights = p.weights,
                    zeros = p.zeros.union({k}), ones = p.ones)
                queue.append(p2)
    return 'Optimal', best.lb, best.xlb

In [9]:
capacity = 15
items = {1,2,3,4,5}
c = {1:50, 2:40, 3:10, 4:70, 5:55}
w = {1:7, 2:5, 3:1, 4:9, 5:6}

res = KnapsackProblemSolve(capacity=capacity,
                     items=items, costs=c, weights=w)
print('Optimal value = ', res[1])
print('Optimal solution = ', res[2])

Optimal value =  125
Optimal solution =  {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
